In [4]:
import sqlite3
import pandas as pd

In [5]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [6]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [7]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [8]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [9]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [10]:
pd.read_sql(
    """
    select * from
        (SELECT count(crim) as crim FROM boston b where b.crim is null),
        (SELECT count(zn) as zn FROM boston b where b.zn is null),
        (SELECT count(indus) as indus FROM boston b where b.indus is null),
        (SELECT count(chas) as chas FROM boston b where b.chas is null),
        (SELECT count(nox) as nox FROM boston b where b.nox is null)
    """,
    con,
)

# получается, что пустых значение нет.

,crim,zn,indus,chas,nox
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [11]:
pd.read_sql(
    """
    SELECT count(distinct crim) as crim,
            count(distinct zn) as zn,
            count(distinct indus) as indus,
            count(distinct chas) as chas,
            count(distinct nox) as nox
    FROM boston b

    """,
    con,
)

,crim,zn,indus,chas,nox
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [12]:
pd.read_sql(
    """
    select
      case when sum(case when a.crim = a.min_crim then 1. else 0 end) / count(*) >= 0.5 then 1 else 0 end as flag_crim,      
      case when sum(case when a.zn = a.min_zn then 1. else 0 end) / count(*) >= 0.5 then 1 else 0 end as flag_zn,      
      case when sum(case when a.indus = a.min_indus then 1. else 0 end) / count(*) >= 0.5 then 1 else 0 end as flag_indus,      
      case when sum(case when a.chas = a.min_chas then 1. else 0 end) / count(*) >= 0.5 then 1 else 0 end as flag_chas,   
      case when sum(case when a.nox = a.min_nox then 1. else 0 end) / count(*) >= 0.5 then 1 else 0 end as flag_nox    
    from 
       (SELECT b.crim,b.zn, b.indus, b.chas, b.nox,
         min(b.crim) over() as min_crim,
         min(b.zn) over() as min_zn, min(b.indus) over() min_indus, 
         min(b.chas) over() as min_chas, min(b.nox) over() min_nox
        FROM boston b) a
    """,
    con,
)

,flag_crim,flag_zn,flag_indus,flag_chas,flag_nox
0,0,1,0,1,0


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [13]:
pd.read_sql(
    """
    select avg(case when medv = max_medv then rm else null end) avg_max,
    avg(case when row <= 25 then rm else null end) a25,
    avg(case when row <= 50 then rm else null end) a50,
    avg(case when row <= 100 then rm else null end) a100,
    avg(case when row <= 200 then rm else null end) a200,
    avg(case when row <= 300 then rm else null end) a300
    from
    (
    SELECT b.rm, b.medv, max(b.medv) over() as max_medv, row_number() over(order by b.medv) as row
    FROM boston B
    --order by b.medv
    ) a
    """,
    con,
)

# влияет ли кол-во комнат на стоимость и как сильно. - влияет примерно 1.5 к 10

,avg_max,a25,a50,a100,a200,a300
0,7.484,5.74784,5.75324,5.88712,5.911705,5.972227


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [14]:
pd.read_sql(
    """
    select lstat, medv, rnk_lstat
    from
    (
    SELECT lstat, medv, rank() over(order by lstat) rnk_lstat, max(medv) over() max_medv
    FROM boston
    )a
    where medv = max_medv
    """,
    con,
)

#  в дорогих домах минимальный процент нищебродов

,lstat,medv,rnk_lstat
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [15]:
pd.read_sql(
    """
    SELECT avg(b.medv) FROM boston b
    where b.chas = 1
    limit 10
    """,
    con,
)

,avg(b.medv)
0,28.44


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [16]:
pd.read_sql(
    """
    SELECT chas, avg(CRIM), avg(ZN), avg(INDUS), avg(NOX)
    FROM boston
    group by 1
    order by 1
    """,
    con,
)

,CHAS,avg(CRIM),avg(ZN),avg(INDUS),avg(NOX)
0,0.0,3.723215,11.634820,11.019193,0.551817
1,1.0,1.851670,7.714286,12.719143,0.593426


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [17]:
pd.read_sql(
    """
    select 'indus', pr_indus*10 as pr, max(indus) as val
    from
        (SELECT indus, ntile(10) over(order by indus) as pr_indus
        FROM boston) a
    group by 2
    union all
    select 'nox', pr_nox*10 as pr, max(nox) as val
    from
        (SELECT nox, ntile(10) over(order by nox) as pr_nox
        FROM boston) a
    group by 2
    """,
    con,
)

,'indus',pr,val
0,indus,10,2.890
1,indus,20,4.390
2,indus,30,5.960
3,indus,40,7.380
4,indus,50,9.690
5,indus,60,13.890
6,indus,70,18.100
7,indus,80,18.100
8,indus,90,19.580
9,indus,100,27.740
